Time series prediction for food sustainability is a ML project 
designed to predict when countries would endure food/forest product shortage
due to environmental factors such as emission

The project follows the steps:
    1. Reading datasets from Amazon S3 buckets
    2. Preprocessing the dataset - Removing NaN, Dropping redundant values, pivot tables etc
    3. Perform Granger causality test
    4. Check for stationary series - ADF test
    5. Perform 1st and 2nd differencing to modify non-stationary series to stationary one
    6. Train dataset with VAR model (each item is trained against different emissions as a separate model)
    7. Run inference with trained model
    8. With the forecast output - perform invert transformation to get back values in original scale
    9. Find the top-k items that endure shortage by particular year
    10. Visualize the production graphs for top-k items

In [1]:
import io
import boto3
import pandas as pd

def read_datasets():
    # Read three datasets stored in the Amazon S3 bucket
    bucket = "faostat-ml"
    file_name = "filtered_files/Emissions_Totals_E_All_Data_(Normalized).csv"
    s3_client = boto3.client("s3")
    obj = s3_client.get_object(Bucket=bucket, Key=file_name)
    df_emission = pd.read_csv(io.BytesIO(obj['Body'].read()))
    df_emission.drop(df_emission.columns[df_emission.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

    #fix the feature names
    df_emission = df_emission.rename(columns={'Item': 'EmissionItem','Value': 'EmissionValue','Element': 'EmissionElement','Unit': 'EmissionUnit'})
    print(df_emission.head(5))

    file_name = "filtered_files/Production_Crops_Livestock_E_All_Data_(Normalized).csv"
    obj = s3_client.get_object(Bucket=bucket, Key=file_name)
    df_prod = pd.read_csv(io.BytesIO(obj['Body'].read()))
    df_prod.drop(df_prod.columns[df_prod.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
    df_prod.drop('Unit', axis=1, inplace=True)
    print(df_prod.head(5))


    file_name = "filtered_files/Forestry_E_All_Data_(Normalized).csv"
    obj = s3_client.get_object(Bucket=bucket, Key=file_name)
    df_forest = pd.read_csv(io.BytesIO(obj['Body'].read()),encoding='latin-1')
    df_forest.drop(df_forest.columns[df_forest.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
    print(df_forest.head(5))
    return df_emission, df_prod, df_forest

In [2]:
def pre_processing():
    remove_rows_df = pre_process_df.copy()
    
    # Removes area, source and emission unit that place no significant role
    remove_rows_df.drop('Area', axis=1, inplace=True)
    remove_rows_df.drop('Source', axis=1, inplace=True)
    remove_rows_df.drop('EmissionUnit', axis=1, inplace=True)
    
    # Remove redudant instance
    remove_rows_df = remove_rows_df[remove_rows_df.Element != "Area harvested"]
    remove_rows_df = remove_rows_df[remove_rows_df.Element != "Production"]
    remove_rows_df = remove_rows_df[remove_rows_df.Element != "Producing Animals/Slaughtered"]
    remove_rows_df = remove_rows_df[remove_rows_df.EmissionElement != "Indirect emissions (N2O)"]
    remove_rows_df = remove_rows_df[remove_rows_df.EmissionElement != "Direct emissions (N2O)"]
    remove_rows_df.index = remove_rows_df.Year
    remove_rows_df.drop('Year', axis=1, inplace=True)

    #identify partial string to look for
    discard = ['from']
    remove_rows_df = remove_rows_df[~remove_rows_df.EmissionElement.str.contains('|'.join(discard))]

    # Combine two features
    remove_rows_df["Emission"] = remove_rows_df["EmissionItem"] + str("_") + remove_rows_df["EmissionElement"]
    remove_rows_df.drop('EmissionItem', axis=1, inplace=True)
    remove_rows_df.drop('EmissionElement', axis=1, inplace=True)

    # Remove redudant instance
    remove_rows_df = remove_rows_df[remove_rows_df.Element != 'Import Value']
    remove_rows_df = remove_rows_df[remove_rows_df.Element != 'Export Value']
    
    emission_list = list(pre_process_df.EmissionItem.unique())

    # Create pivot table for production items + forestry products based on year and item
    df_item = remove_rows_df.pivot_table(index=['Year'], 
                columns=['Item'], values='Value')
    
    # Remove columns that have atleast one NaN value since it would affect the forecast
    sum = df_item.isnull().sum(axis = 0)
    for items in sum.iteritems():
        if(items[1]>0):
            df_item.drop(items[0], axis=1, inplace=True)
    nan_cols = [i for i in df_item.columns if df_item[i].isnull().any()]

    # Create pivot table for emissions
    df_emi = remove_rows_df.pivot_table(index=['Year'], 
                columns=['Emission'], values='EmissionValue')
    # Remove columns that have atleast one NaN value since it would affect the forecast
    sum = df_emi.isnull().sum(axis = 0)
    for items in sum.iteritems():
        if(items[1]>0):
            df_emi.drop(items[0], axis=1, inplace=True)
    nan_cols = [i for i in df_emi.columns if df_emi[i].isnull().any()]
    
    display(df_item.head(5))
    display(df_emi.head(5))
    return df_item, df_emi, emission_list

In [3]:
# Perform granger causation test - to check the influence of one variable over another

def grangers_causation_matrix(data, variables, test='ssr_chi2test', verbose=False):    
    from statsmodels.tsa.stattools import grangercausalitytests
    maxlag=12
    """Check Granger Causality of all possible combinations of the Time series.
    The rows are the response variable, columns are predictors. The values in the table 
    are the P-Values. P-Values lesser than the significance level (0.05), implies 
    the Null Hypothesis that the coefficients of the corresponding past values is 
    zero, that is, the X does not cause Y can be rejected.
    """
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    return df

In [4]:
# Perform ADF test to check if each series is stationary or not

def adfuller_test(series, signif=0.05, name='', verbose=False):
    
    from statsmodels.tsa.api import VAR
    from statsmodels.tsa.stattools import adfuller
    from statsmodels.tools.eval_measures import rmse, aic
    non_stationary = []
    """Perform ADFuller to test for Stationarity of given series and print report"""
    r = adfuller(series, autolag='AIC')
    output = {'test_statistic':round(r[0], 4), 'pvalue':round(r[1], 4), 'n_lags':round(r[2], 4), 'n_obs':r[3]}
    p_value = output['pvalue'] 
    def adjust(val, length= 6): return str(val).ljust(length)

    # Print Summary
    print(f'    Augmented Dickey-Fuller Test on "{name}"', "\n   ", '-'*47)
    print(f' Null Hypothesis: Data has unit root. Non-Stationary.')
    print(f' Significance Level    = {signif}')
    print(f' Test Statistic        = {output["test_statistic"]}')
    print(f' No. Lags Chosen       = {output["n_lags"]}')

    for key,val in r[4].items():
        i = i + 1
        print(f' Critical value {adjust(key)} = {round(val, 3)}')

    if p_value <= signif:
        print(f" => P-Value = {p_value}. Rejecting Null Hypothesis.")
        print(f" => Series is Stationary.")
    else:
        print(f" => P-Value = {p_value}. Weak evidence to reject the Null Hypothesis.")
        print(f" => Series is Non-Stationary.")

In [5]:
# Revert back the differencing to get the forecast to original scale

def invert_transformation(df_train, df_forecast, second_diff=False):
    df_fc = df_forecast.copy()
    columns = df_train.columns
    for col in columns:        
        # Roll back 2nd Diff
        if second_diff:
            df_fc[str(col)+'_1d'] = (df_train[col].iloc[-1]-df_train[col].iloc[-2]) + df_fc[str(col)+'_2d'].cumsum()
        # Roll back 1st Diff
        df_fc[str(col)+'_forecast'] = df_train[col].iloc[-1] + df_fc[str(col)+'_1d'].cumsum()
    return df_fc

In [6]:
# Perform stationary test and differencing

def stationary(df):    
    df_train, df_test = pandas_df[0:-nobs], pandas_df[-nobs:]
    # 1st difference
    df_differenced = df_train.diff().dropna()
    # Second Differencing
    df_differenced = df_differenced.diff().dropna()   
    return df_differenced, df_train, df_test

In [7]:
# The function that performs forecasting - training and inference at the same time

def forecasting(df_item, df_emi, emission_list):
    import warnings
    import pandas as pd
    import plotly.graph_objs as go
    from statsmodels.tsa.api import VAR
    warnings.simplefilter(action='ignore', category=FutureWarning)
    mean = []
    top_k = topk
    shortage = []
    top_k_items = dict()
    nobs = forecast_year
    col = df_item.columns
    pandas_df = pd.DataFrame()
    df_graphs_forcast = pd.DataFrame()
    df_graphs_historical = pd.DataFrame()

    
    # iterate through every item - production and forestry product - for the country/area specified
    for i in col:
        selected_columns = df_item[i]

        new_df = selected_columns.copy()
        new_merged_df = pd.merge(new_df, df_emi, on=['Year'])

        for j in emission_list:
            emission = new_merged_df.filter(regex=j)
            col = list(emission.columns) 
            new_merged_df[j] = new_merged_df[col].sum(axis=1)

        pandas_df = new_merged_df[new_merged_df.columns.intersection(emission_list)]
        pandas_df[str(new_merged_df.columns[0])] = new_merged_df[str(new_merged_df.columns[0])]
        pandas_df = pandas_df.loc[:, (pandas_df != 0).any(axis=0)]
        
        # removing features that do not become stationary even after 2nd differencing
        to_remove = ['Enteric Fermentation', 'Manure left on Pasture']
        pandas_df.drop(to_remove, axis=1, inplace=True)
        
        # Granger's causality test
        grangers_causation_matrix(pandas_df, variables = pandas_df.columns)         
        
        df_train, df_test = pandas_df[0:-forecast_year], pandas_df[-forecast_year:]
        # 1st difference
        df_differenced = df_train.diff().dropna()
        # 2nd Differencing
        df_differenced = df_differenced.diff().dropna()   
    
        # Tarin VAR model with lag order - 6
        model = VAR(df_differenced)
        model_fitted = model.fit(6, trend="ct")

        # Get the lag order
        lag_order = model_fitted.k_ar

        # Input data for forecasting
        forecast_input = df_differenced.values[-lag_order:]
        fc = model_fitted.forecast(y=forecast_input, steps=forecast_year)
        df_forecast = pd.DataFrame(fc, index=pandas_df.index[-forecast_year:], columns=pandas_df.columns + '_2d')

        
        # Perform invert transformation to get values in original scale
        df_results = invert_transformation(df_train, df_forecast, second_diff=True)

        d = pandas_df.tail(forecast_year)
        d.reset_index(inplace = True)

        # Create dataframe with the forecasted value by adjusting the year
        range_year = pd.date_range(start = str(d.Year.iloc[-1]), periods = (len(d)+1), freq = 'A')
        range_year = range_year.year
        year = pd.DataFrame({'Year': range_year})
        d = d.append(year)
        year_forecast = d['Year'].iat[-1]

        d.set_index('Year', inplace = True)
        d = d.tail(forecast_year)
        df_results.index = d.index
        
        # Skip certain items that have null values
        if i =='Fibre Crops Primary':
            continue
            
        # Calculate mean of historical values
        mean_old = pandas_df[i].mean()
        
        recent_forecast = df_results[i+"_forecast"]
        recent_forecast = recent_forecast.iat[-1]

        # Test the condition - recent values that are 50% less than historical values are more likely to have shortage
        condition = ((150/100)*mean_old)
        if(recent_forecast < condition):
            shortage.append(i)
            diff = abs(recent_forecast - mean_old)
            top_k_items[i] = diff
            fig = go.Figure()
            n = df_results.index[0]
            df_graphs_historical['historical_values_'+i] = pandas_df[str(i)]
            df_graphs_forcast['forecasted_values_'+i] = df_results[i+"_forecast"]
        
    # Sort and list top-k products from the dict        
    find_top_k = sorted(top_k_items.keys(), reverse=True)[:top_k]
    print("Top -",top_k, "products that will face shortage by",year_forecast)
    print("==============================================")
    for i in range(len(find_top_k)):
        print(find_top_k[i])   
    
    # Plot graphs for the top-k items
    for i in range(len(find_top_k)):
        fig = go.Figure()
        fig.add_trace(go.Scatter(x = df_graphs_historical.index, y = df_graphs_historical["historical_values_" +str(find_top_k[i])], marker = dict(color ="red"), name = "Actual close price"))
        fig.add_trace(go.Scatter(x = df_graphs_forcast.index, y = df_graphs_forcast["forecasted_values_" +str(find_top_k[i])], marker=dict(color = "green"), name = "Future prediction"))
        fig.update_xaxes(showline = True, linewidth = 2, linecolor='black', mirror = True, showspikes = True,)
        fig.update_yaxes(showline = True, linewidth = 2, linecolor='black', mirror = True, showspikes = True,)
        fig.update_layout(title= str(forecast_year) +" Years Forecast", yaxis_title = find_top_k[i], hovermode = "x", hoverdistance = 100) #, # Distance to show hover label of data point spikedistance = 1000,shapes = [dict( x0 = n, x1 = n, y0 = 0, y1 = 1, xref = 'x', yref = 'paper', line_width = 2)], annotations = [dict(x = n, y = 0.05, xref = 'x', yref = 'paper', showarrow = False, xanchor = 'left', text = 'Prediction')])
        fig.update_layout(autosize = False, width = 1000, height = 400,)
        fig.show()

In [8]:
def time_series_prediction():
    pre_process_df = pd.DataFrame()
    combined_df = pd.DataFrame()

    import numpy as np
    import seaborn as sb
    import ipywidgets as widgets
    import matplotlib.pyplot as plt
    from sklearn import preprocessing
    from sklearn.preprocessing import LabelEncoder
    from ipywidgets import Layout, Button, Box, FloatText, Textarea, Dropdown, Label, IntSlider

    def unique_sorted_values_plus_ALL(array):
        unique = array.unique().tolist()
        unique.sort()
        return unique
    output_area = widgets.Output()
    print("Area:")
    dropdown_area = widgets.Dropdown(options=unique_sorted_values_plus_ALL(df_prod.Area))
    def dropdown_area_eventhandler(change):
        output_area.clear_output()
        with output_area:
            if(change.new):
                country_df_prod = df_prod[df_prod.Area == change.new]
                country_df_forest = df_forest[df_forest.Area == change.new]
                country_df_emission = df_emission[df_emission.Area == change.new]

                prod_forest_df = pd.concat([country_df_prod, country_df_forest], ignore_index=True)
                global pre_process_df, combined_df
                pre_process_df = pd.merge(prod_forest_df, country_df_emission, on=['Year','Area'])
    dropdown_area.observe(dropdown_area_eventhandler, names='value')
    display(dropdown_area)

    print("Number of years ahead to forecast:")
    forecast_year = 0
    output_year = widgets.Output()

    dropdown_year = widgets.Dropdown(options=range(5,16))
    def dropdown_forecastyear_eventhandler(change_year):
        output_year.clear_output()
        with output_year:
            if(change_year.new):
                global forecast_year
                forecast_year = change_year.new
    dropdown_year.observe(dropdown_forecastyear_eventhandler, names='value')
    display(dropdown_year)

    print("k items that endure shortage:")
    topk = 0
    output_topk = widgets.Output()

    dropdown_topk = widgets.Dropdown(options=range(2,11))
    def dropdown_topk_eventhandler(change_topk):
        output_topk.clear_output()
        with output_topk:
            if(change_topk.new):
                global topk
                topk = change_topk.new
    dropdown_topk.observe(dropdown_topk_eventhandler, names='value')
    display(dropdown_topk)
    display(output_year)

In [9]:
# Run the functions from this module onwards for time series prediction
df_emission, df_prod, df_forest = read_datasets()

          Area          EmissionItem  EmissionElement  Year      Source  \
0  Afghanistan  Enteric Fermentation  Emissions (CH4)  1961  FAO TIER 1   
1  Afghanistan  Enteric Fermentation  Emissions (CH4)  1962  FAO TIER 1   
2  Afghanistan  Enteric Fermentation  Emissions (CH4)  1963  FAO TIER 1   
3  Afghanistan  Enteric Fermentation  Emissions (CH4)  1964  FAO TIER 1   
4  Afghanistan  Enteric Fermentation  Emissions (CH4)  1965  FAO TIER 1   

  EmissionUnit  EmissionValue  
0   kilotonnes       240.6831  
1   kilotonnes       245.3106  
2   kilotonnes       255.8285  
3   kilotonnes       259.0650  
4   kilotonnes       265.5980  
          Area                 Item         Element  Year   Value
0  Afghanistan  Almonds, with shell  Area harvested  1975     0.0
1  Afghanistan  Almonds, with shell  Area harvested  1976  5900.0
2  Afghanistan  Almonds, with shell  Area harvested  1977  6000.0
3  Afghanistan  Almonds, with shell  Area harvested  1978  6000.0
4  Afghanistan  Almonds, wi

In [10]:
time_series_prediction()

Area:


Dropdown(options=('Afghanistan', 'Africa', 'Albania', 'Algeria', 'Americas', 'Angola', 'Antigua and Barbuda', …

Number of years ahead to forecast:


Dropdown(options=(5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15), value=5)

k items that endure shortage:


Dropdown(options=(2, 3, 4, 5, 6, 7, 8, 9, 10), value=2)

Output()

In [14]:
import warnings
warnings.filterwarnings('ignore')
warnings.warn('DelftStack')
warnings.warn('Do not show this message')

display(pre_process_df.head(5))
df_item, df_emi, emission_list = pre_processing()
forecasting(df_item, df_emi, emission_list)

,Area,Item,Element,Year,Value,EmissionItem,EmissionElement,Source,EmissionUnit,EmissionValue
0,United States of America,"Almonds, with shell",Area harvested,1961,36138.0,Enteric Fermentation,Emissions (CH4),FAO TIER 1,kilotonnes,6876.7867
1,United States of America,"Almonds, with shell",Area harvested,1961,36138.0,Enteric Fermentation,Emissions (CO2eq) from CH4 (AR5),FAO TIER 1,kilotonnes,192550.0266
2,United States of America,"Almonds, with shell",Area harvested,1961,36138.0,Enteric Fermentation,Emissions (CO2eq) (AR5),FAO TIER 1,kilotonnes,192550.0266
3,United States of America,"Almonds, with shell",Area harvested,1961,36138.0,Manure Management,Emissions (CH4),FAO TIER 1,kilotonnes,1583.0819
4,United States of America,"Almonds, with shell",Area harvested,1961,36138.0,Manure Management,Emissions (N2O),FAO TIER 1,kilotonnes,46.3383


Item,"Almonds, with shell",Apples,Apricots,Artichokes,Asparagus,Asses,Avocados,Bananas,Barley,"Beans, dry",...,Vegetables Primary,"Vegetables, fresh nes",Veneer sheets,"Walnuts, with shell",Watermelons,Wheat,Wood charcoal,Wood pulp,Wood residues,Wood-based panels
Year,,,,,,,,,,,,,,,,,,,,,
1961,16669.0,139842.0,107146.0,67247.0,28027.0,15000.0,48990.0,105447.0,16489.0,15413.0,...,132949.0,234949.0,110650.0,10633.0,105983.0,16070.0,6750.0,428350.0,279800.0,292100.0
1962,11558.0,141510.0,96730.0,61656.0,28572.0,15000.0,44532.0,97139.0,18839.0,14049.0,...,143092.0,244999.0,148150.0,12543.0,108828.0,16809.0,6050.0,492700.0,277300.0,382250.0
1963,13684.0,143045.0,115581.0,67260.0,29000.0,15000.0,50850.0,91981.0,18808.0,16153.0,...,138346.0,240051.0,170950.0,13221.0,121760.0,16949.0,6650.0,512300.0,252100.0,447150.0
1964,16517.0,156929.0,124560.0,72914.0,27284.0,16000.0,31137.0,114667.0,20209.0,13886.0,...,139054.0,239961.0,207850.0,14120.0,110228.0,17344.0,9900.0,545900.0,254300.0,531100.0
1965,13932.0,158906.0,132615.0,78461.0,28015.0,16000.0,57109.0,84583.0,23071.0,12302.0,...,143821.0,245049.0,235150.0,12811.0,115887.0,17853.0,9350.0,528950.0,327600.0,583950.0


Emission,Agricultural Soils_Emissions (CO2eq) (AR5),Agricultural Soils_Emissions (N2O),Burning - Crop residues_Emissions (CH4),Burning - Crop residues_Emissions (CO2eq) (AR5),Burning - Crop residues_Emissions (N2O),Crop Residues_Emissions (CO2eq) (AR5),Crop Residues_Emissions (N2O),Enteric Fermentation_Emissions (CH4),Enteric Fermentation_Emissions (CO2eq) (AR5),IPCC Agriculture_Emissions (CH4),...,Manure Management_Emissions (CO2eq) (AR5),Manure Management_Emissions (N2O),Manure applied to Soils_Emissions (CO2eq) (AR5),Manure applied to Soils_Emissions (N2O),Manure left on Pasture_Emissions (CO2eq) (AR5),Manure left on Pasture_Emissions (N2O),Rice Cultivation_Emissions (CH4),Rice Cultivation_Emissions (CO2eq) (AR5),Synthetic Fertilizers_Emissions (CO2eq) (AR5),Synthetic Fertilizers_Emissions (N2O)
Year,,,,,,,,,,,,,,,,,,,,,
1961,72179.6003,272.3759,86.7807,3026.0759,2.2499,10533.5156,39.7491,6876.7867,192550.0266,8771.6993,...,56605.9467,46.3383,12387.5891,46.7456,32949.9135,124.3393,225.0500,6301.400,16308.5820,61.5418
1962,75376.9699,284.4414,81.3856,2837.9454,2.1100,10325.3373,38.9635,6980.1400,195443.9200,8946.6938,...,58279.7363,47.2514,12405.3478,46.8126,33902.3107,127.9332,250.9500,7026.600,18743.9741,70.7320
1963,79799.0153,301.1284,86.0761,3001.5042,2.2316,10919.0427,41.2039,7136.3680,199818.3040,9093.2366,...,58135.8224,48.2273,12385.9309,46.7394,35405.5323,133.6058,250.9500,7026.600,21088.5094,79.5793
1964,82230.4558,310.3036,83.8071,2922.3852,2.1728,10446.0853,39.4192,7251.5884,203044.4759,9177.3664,...,57471.1383,48.9862,12387.4285,46.7450,36650.3759,138.3033,253.0500,7085.400,22746.5661,85.8361
1965,86771.0895,327.4381,83.6993,2918.6253,2.1700,11598.6454,43.7685,7253.3471,203093.7178,9084.8934,...,54726.9855,48.6675,12209.2005,46.0725,37438.0311,141.2756,253.9145,7109.606,25525.2126,96.3216


Top - 10 products that will face shortage by 2024
Wood residues
Wheat
Watermelons
Walnuts, with shell
Veneer sheets
Vegetables, fresh nes
Vegetables Primary
Turkeys
Treenuts, Total
Taro (cocoyam)
